# Loss Function: 

La Loss Function o Cost Function es la funcion que elijamos para calcular cómo está performando nuestro modelo. Esta función mapea los valores de las variables del modelo con un número real que representa el "costo" que esa version del modelo tiene respecto a los valores desables. Idealmente, el objetivo del modelo es optimizar el valor de los weights and biases hasta obtener el costo más bajo, para ello, 

Intuitivamente diría que esta función se utiliza con modelos supervisado porque entre sus parametros recibe los targets o labels del modelo, pero esto habría que revisarlo mejor.

En la bibliografía e incluso en pytorch, esta función recibe distintos nombre:

- error function
- criterion function
- cost function
- objective function
- loss function

Si bien, por regla general todas la denominaciones hacen referencia a lo mismo, esto no es estrictamente así en todos los escenarios.

Una diferencia entre funciones cost, loss y objective esta en este [comentario de stack exchange](https://stats.stackexchange.com/questions/179026/objective-function-cost-function-loss-function-are-they-the-same-thing)

La noción de 'criterion' y la idea general de función loss que viene del decision theory se me escapa bastante.


-------------------------------------------------
<sub>To do: explicar estas diferencias de denominación</sub>

## Por qué funciona?

Ok, pero cómo saber qué cambios debemos hacer en nuestros w&b, que recordemos se inicializaorn con valores random, para obtener el resultado más bajo de nuestra función loss? 

Dejemos que alguien lo explique mejor, como, por ejemplo, este [post de kaggle](https://www.kaggle.com/erdemuysal/linear-regression-from-scratch-and-with-pytorch)

"The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

<img src="LOSSIMAGE.png">


The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model."


# Distintas Loss Functions y su aplicación

## Problemas de regresión 

Para problemas de regresión[<sup>1</sup>](#fn1), una de las funciones loss más usadas es el Mean Squared Error, pero también pueden aplicarse  Mean Absolute Error u otras y su elección depende de la distribución de los datos, la presencia de outliers, factores que habría que analizar al momento de tener un corpus.


### Mean Squared Error Loss

Esta función toma los valores esperados (y) y los valores de la prediccion (y_hat) y suma el total de sus distancias cuadradas. El total de esta suma es promediado por el numero de ejemplos.


<img src="MSEL.png">

----------------------------------------------------------------------------------------------------------------
<sub>[<sup id="fn1">1</sup>](#fn1-back)Un problema de regresión es aquel cuyas variables de salida (o outputs) son valores reales o continuos, como 'salario', 'peso', 'costo de una propiedad'.</span></sub>


In [98]:
#Implementación en Pytorch

import torch
import torch.nn as nn

mse_loss = nn.MSELoss()
outputs = torch.randn(3, 5, requires_grad=True)
targets = torch.randn(3, 5)
loss = mse_loss(outputs, targets)
print('output: ', outputs)
print('targets: ', targets)
print('-------------------')
#print('loss: ', loss)

output:  tensor([[ 0.7359,  0.0151, -0.5575, -0.0653,  1.9399],
        [-0.0838, -1.0402, -1.4656,  0.7395, -0.9094],
        [-1.3699,  0.0111, -0.3795, -0.0299,  0.2474]], requires_grad=True)
targets:  tensor([[-0.2327, -0.5158, -1.0786, -1.4120, -1.2959],
        [ 1.3439,  0.0142,  0.3374, -1.5199,  0.4060],
        [ 1.9622,  0.0635, -1.2803, -0.1006, -0.4490]])
-------------------


In [99]:
# Cuenta propia para ver cómo funciona

sum_se = torch.zeros(5)
print('sum_se', sum_se)
for i in zip(targets, outputs):
    se = (i[0] - i[1]) ** 2
    print('se', se)
    sum_se += se
    print('sum_se', sum_se)
print('end', sum_se)
    
sum_num = torch.sum(sum_se)
print(sum_num)

mse = sum_num / 15
#mse = sum_num / len(targets) descomentar para ver el resultado de interpretar N como len de targets en vez de MxN
print(mse)

sum_se tensor([0., 0., 0., 0., 0.])
se tensor([ 0.9381,  0.2819,  0.2716,  1.8137, 10.4705], grad_fn=<PowBackward0>)
sum_se tensor([ 0.9381,  0.2819,  0.2716,  1.8137, 10.4705], grad_fn=<AddBackward0>)
se tensor([2.0383, 1.1118, 3.2508, 5.1047, 1.7302], grad_fn=<PowBackward0>)
sum_se tensor([ 2.9764,  1.3937,  3.5224,  6.9185, 12.2007], grad_fn=<AddBackward0>)
se tensor([1.1103e+01, 2.7474e-03, 8.1147e-01, 5.0023e-03, 4.8500e-01],
       grad_fn=<PowBackward0>)
sum_se tensor([14.0795,  1.3964,  4.3338,  6.9235, 12.6857], grad_fn=<AddBackward0>)
end tensor([14.0795,  1.3964,  4.3338,  6.9235, 12.6857], grad_fn=<AddBackward0>)
tensor(39.4189, grad_fn=<SumBackward0>)
tensor(2.6279, grad_fn=<DivBackward0>)


### Por qué 15?

Cuando tomé la primera descripción de la función 'no me daban los números' y mi resultado no era el mismo que el de aplicar la función de pytorch, hasta que encontré esta descripción que hace explicito que los valores involucrados no son puntos sino matrices mxn.

<img src="MSELMxN.png">

### Linear Regression with california housing dataset (default available in colab)

Con este ejemplo quise ilustrar de manera simple el uso de MSE en un modelo entrenado con pytorch. De ningún modo este modelo funciona, ni en este ni en ningún otro data set de regresión, pero se podria acomodar para que lo haga, empezando por agregar un DataLoader.

In [100]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [101]:
LR = 0.01 
EPOCHS = 3

In [102]:
train_df = pd.read_csv('./california_housing_train.csv')
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [103]:
#clean empty values and inf
train_df.replace([np.inf, -np.inf], np.nan)  # Replace inf values with NaNs
train_df.dropna(inplace=True)  # Drop NaN values from the dataframe

In [104]:
#inputs, targets
# Convert from Pandas dataframe to numpy arrays
inputs = train_df.drop(['median_house_value'], axis=1).to_numpy()  # Drop TARGET_COLUMN since it is target, can not be used as input
targets = train_df[['median_house_value']].to_numpy()# Drop 'ocean_proximity' since it is not numerical value
inputs.shape, targets.shape

((17000, 8), (17000, 1))

In [105]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [106]:
class linearRegression(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        out = self.linear(x)
        return out

In [107]:
model = linearRegression(8, 1) #in_features = 8dim, out_features=1dim

In [108]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [109]:
for epoch in range(EPOCHS):
    

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    print('input: ', inputs[0])
    outputs = model(inputs)
    print('output: ', outputs[0])
    print('target: ', targets[0])

    # get loss for the predicted output
    loss = criterion(outputs, targets)
    #print(loss)
        
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))
    print('---------------------------------------------')

input:  tensor([-1.1431e+02,  3.4190e+01,  1.5000e+01,  5.6120e+03,  1.2830e+03,
         1.0150e+03,  4.7200e+02,  1.4936e+00])
output:  tensor([1508.0210], grad_fn=<SelectBackward>)
target:  tensor([66900.])
epoch 0, loss 56049602560.0
---------------------------------------------
input:  tensor([-1.1431e+02,  3.4190e+01,  1.5000e+01,  5.6120e+03,  1.2830e+03,
         1.0150e+03,  4.7200e+02,  1.4936e+00])
output:  tensor([7.4670e+10], grad_fn=<SelectBackward>)
target:  tensor([66900.])
epoch 1, loss 2.7698533824213035e+21
---------------------------------------------
input:  tensor([-1.1431e+02,  3.4190e+01,  1.5000e+01,  5.6120e+03,  1.2830e+03,
         1.0150e+03,  4.7200e+02,  1.4936e+00])
output:  tensor([-2.3301e+16], grad_fn=<SelectBackward>)
target:  tensor([66900.])
epoch 2, loss 2.7146369316725845e+32
---------------------------------------------


### Categorical Cross-Entropy Loss 

Categorical-cross Entropy loss es usualmente aplicado en problemas de clasificación multi clase donde el output del modelo es interpretado como la probabilidad de que un dato pertenezca a una clase.

<img src="images.png">


In [132]:
import torch
import torch.nn as nn

ce_loss = nn.CrossEntropyLoss()
outputs = torch.randn(3, 5, requires_grad=True)
targets = torch.tensor([1, 0, 3], dtype=torch.int64)
loss = ce_loss(outputs, targets)
print(loss)

tensor(1.3731, grad_fn=<NllLossBackward>)


In [133]:
print(outputs[0])
print(targets[0])

tensor([-1.9297,  0.6877,  1.7151,  0.8076, -0.9618], grad_fn=<SelectBackward>)
tensor(1)


In [140]:
#To-do: cuenta manual
# Cuenta propia para ver cómo funciona

result = torch.zeros(5)
for i in zip(targets, outputs):
    result += i[0] * torch.log(i[1])
    
sum_res = torch.sum(result)
print(sum_res)

tensor(nan, grad_fn=<SumBackward0>)


### Clasificación Multi Clase con MNIST dataset (default available in colab)

In [87]:
train_df = pd.read_csv('./mnist_train_small.csv', header=None, index_col=None)
print(train_df.head())
print('length columns: ', len(train_df.columns)) #28px x 28px = 784 + label = 1 --> 785 columns
train_df.columns[0], train_df.columns[300], train_df.columns[784] #??

   0    1    2    3    4    5    6    7    8    9    ...  775  776  777  778  \
0    6    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1    5    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
2    7    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    9    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    5    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   779  780  781  782  783  784  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 785 columns]
length columns:  785


(0, 300, 784)

In [88]:
#inputs, labels
# Convert from Pandas dataframe to numpy arrays
inputs = train_df.drop([0], axis=1).to_numpy()  
labels = train_df[0].to_numpy()
inputs.shape, labels_col.shape

((20000, 784), (20000,))

In [89]:
labels_col[0], labels_col[18], labels_col[1880]

(6, 7, 8)

In [90]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
labels = torch.from_numpy(labels)
labels = labels.type(torch.LongTensor) #la función cross-entropy espera un valor representando la categoria

In [91]:
class Classification(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(Classification, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        out = self.linear(x)
        return out

In [95]:
model = Classification(784,10)

In [96]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [97]:
for epoch in range(EPOCHS):
    

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    #print('input: ', inputs[0])
    outputs = model(inputs)
    print('labels: ', labels[0])
    loss_outputs = torch.sigmoid(outputs)
    #print('output: ', loss_outputs)
    # get loss for the predicted output
    loss = criterion(loss_outputs, labels)
    #print(loss)
        
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))
    print('---------------------------------------------')

labels:  tensor(6)
epoch 0, loss 2.3924241065979004
---------------------------------------------
labels:  tensor(6)
epoch 1, loss 2.302994966506958
---------------------------------------------
labels:  tensor(6)
epoch 2, loss 2.241755962371826
---------------------------------------------


### Binary Cross-Entropy Loss (binary classification, and multilabel)

### Combining Loss Functions for Multi Task Learning problems


<img src="Diet.png">